# 🔄 Cohere Reranker로 검색 결과 품질 향상하기

## 📚 개요

**Cohere Reranker**는 검색 결과를 **더 정확하고 관련성 높게 재정렬**해주는 똑똑한 도구입니다! 🎯

### 🏪 온라인 쇼핑몰로 이해하기

온라인 쇼핑에서 "블루투스 이어폰"을 검색했다고 생각해보세요:

#### 🔍 **일반적인 검색 결과** (기본 Retriever)
```
1. 블루투스 스피커 📻
2. 유선 이어폰 🎧  
3. 블루투스 이어폰 🎵 ← 원하는 상품!
4. 헤드폰 🎧
5. 이어폰 케이스 📦
```

#### ⚡ **Reranker 적용 후**
```
1. 블루투스 이어폰 🎵 ← 가장 관련성 높은 결과가 맨 위로!
2. 무선 이어폰 🎵
3. 에어�팟 유형 이어폰 🎵
4. 블루투스 헤드폰 🎧
5. 블루투스 스피커 📻
```

### 🎯 Cohere Reranker의 핵심 가치

✅ **정확성 향상**: 진짜 원하는 정보를 맨 위에 배치  
✅ **맥락 이해**: 단순한 키워드 매칭을 넘어선 의미 파악  
✅ **다국어 지원**: 한국어, 영어 등 여러 언어 지원  
✅ **간편한 통합**: LangChain과 완벽하게 호환

### 📋 목차

1. **🛠️ 환경 설정**
   - Cohere API 키 설정
   - 필요한 라이브러리 import

2. **🎓 기본 사용법**
   - 문서 로드 및 벡터 저장소 구축
   - 기본 검색 테스트

3. **⚡ Reranker 적용**
   - CohereRerank 설정
   - ContextualCompressionRetriever와 결합
   - 성능 비교 및 결과 분석

### 💡 Cohere란?

**Cohere**는 기업이 인간-기계 상호작용을 개선할 수 있도록 돕는 자연어 처리 모델을 제공하는 캐나다의 AI 스타트업입니다.

- **🌟 특징**: 강력한 다국어 지원과 높은 성능
- **🔗 공식 문서**: [Cohere Rerank API](https://docs.cohere.com/docs/reranking)
- **🏆 장점**: 검색 품질을 획기적으로 개선

이제 실제 예제를 통해 Cohere Reranker의 강력한 성능을 체험해봅시다! 🚀

In [ ]:
# 설치
# !pip install -qU cohere

---

# 🛠️ 환경 설정

## 🔑 Cohere API 키 설정

Cohere Reranker를 사용하려면 **무료 API 키**가 필요합니다!

### 📋 API 키 발급 단계

1. **🌐 Cohere 대시보드 방문**: [API 키 발급 페이지](https://dashboard.cohere.com/api-keys)
2. **📝 계정 생성 또는 로그인**
3. **🔑 새 API 키 생성**
4. **📋 API 키 복사**

### 💾 환경 변수 설정

`.env` 파일에 다음과 같이 API 키를 설정해주세요:

```bash
COHERE_API_KEY=your_api_key_here
```

### 📚 유용한 참고 자료

- 📖 **[공식 문서](https://docs.cohere.com/docs/the-cohere-platform)**: Cohere 플랫폼 전반적인 가이드
- 🤖 **[Reranker 모델 목록](https://docs.cohere.com/docs/rerank-2)**: 사용 가능한 모델들과 성능 비교
- 🌍 **다국어 지원**: `rerank-multilingual-v3.0`, `rerank-multilingual-v2.0` 등

### 💡 팁

> 🆓 **무료 사용량**: Cohere는 한 달에 **1,000회 무료 호출**을 제공합니다!  
> 💰 **비용 효율적**: 검색 품질 향상 대비 매우 경제적인 솔루션입니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

---

# 🎓 기본 사용법

이제 실제로 Cohere Reranker를 사용해보겠습니다!

## 🧰 작업 준비

먼저 문서 출력을 위한 유틸리티 함수를 만들어보겠습니다. 이 함수는 검색 결과를 **깔끔하게 보여주는 역할**을 합니다.

In [ ]:
# 검색 결과를 보기 좋게 출력하는 유틸리티 함수
def pretty_print_docs(docs):
    """
    문서 리스트를 보기 좋은 형태로 출력하는 함수
    각 문서를 번호와 함께 구분선으로 나누어 표시
    """
    # 문서들을 번호와 함께 구분선으로 나누어 출력
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## 🌍 Cohere 다국어 지원 모델

Cohere의 **큰 장점 중 하나는 강력한 다국어 지원**입니다! 특히 한국어도 매우 잘 처리합니다. 🇰🇷

### 🎯 추천 모델 조합

#### **📊 Embedding 모델** (벡터 변환용)
- **`embed-multilingual-v3.0`** ⭐ **추천!**: 최신 모델, 성능 우수
- **`embed-multilingual-light-v3.0`**: 빠른 처리 속도가 필요할 때
- **`embed-multilingual-v2.0`**: 안정성을 원할 때

#### **🔄 Reranker 모델** (재정렬용)
- **`rerank-multilingual-v3.0`** ⭐ **추천!**: 최신 재정렬 모델
- **`rerank-multilingual-v2.0`**: 검증된 안정성

### 💡 모델 선택 가이드

```
🚀 최고 성능이 필요하다면:
   Embedding: embed-multilingual-v3.0
   Reranker: rerank-multilingual-v3.0

⚡ 속도가 중요하다면:
   Embedding: embed-multilingual-light-v3.0  
   Reranker: rerank-multilingual-v3.0

🛡️ 안정성을 우선한다면:
   Embedding: embed-multilingual-v2.0
   Reranker: rerank-multilingual-v2.0
```

### 🌐 지원 언어

한국어, 영어, 중국어, 일본어, 독일어, 프랑스어, 스페인어 등 **100개 이상의 언어**를 지원합니다!

In [ ]:
# 필요한 라이브러리 import
from langchain_community.document_loaders import TextLoader  # 텍스트 파일 로더
from langchain_community.vectorstores import FAISS  # 벡터 저장소
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain_cohere import CohereEmbeddings  # Cohere 임베딩 모델

# 1단계: 문서 로드 - 키워드 설명 파일 불러오기
documents = TextLoader("./data/appendix-keywords.txt").load()

# 2단계: 텍스트 분할기 초기화
# chunk_size=500: 각 조각을 500자로 제한
# chunk_overlap=100: 조각 간 100자씩 겹치게 하여 문맥 유지
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 3단계: 문서를 작은 조각들로 분할
texts = text_splitter.split_documents(documents)

# 4단계: FAISS 벡터 저장소 생성 및 검색기 설정
# Cohere의 다국어 임베딩 모델 사용
# k=10: 상위 10개 문서를 검색
retriever = FAISS.from_documents(
    texts, CohereEmbeddings(model="embed-multilingual-v3.0")
).as_retriever(search_kwargs={"k": 10})

# 5단계: 테스트 질의문 작성
query = "Word2Vec 에 대해서 알려줘!"

# 6단계: 기본 검색 실행 (Reranker 적용 전)
docs = retriever.invoke(query)

# 7단계: 검색 결과 출력 - Reranker 적용 전 결과
print("🔍 기본 검색 결과 (Reranker 적용 전):")
pretty_print_docs(docs)

---

# ⚡ Cohere Reranker 적용

## 🎯 핵심 아이디어

기본 검색 결과를 **Cohere의 똑똑한 AI가 재정렬**해서 더 정확한 결과를 만듭니다!

### 📊 작동 원리

#### 1️⃣ **기본 검색** (FAISS + Cohere Embeddings)
```
사용자 질문 → 벡터 변환 → 유사도 계산 → 상위 10개 선택
```
- **장점**: 빠른 속도
- **단점**: 단순한 벡터 유사도만 고려

#### 2️⃣ **Reranker 적용** (CohereRerank)
```
기본 검색 결과 → AI가 질문과의 관련성 재평가 → 순서 재정렬
```
- **장점**: 맥락과 의미까지 깊이 이해
- **단점**: 약간의 추가 시간 소요

### 🔧 ContextualCompressionRetriever

**ContextualCompressionRetriever**는 **기본 검색기와 재정렬 모델을 연결하는 다리** 역할을 합니다!

- **`base_retriever`**: 1차 검색을 담당 (FAISS)
- **`base_compressor`**: 결과를 개선 (CohereRerank)

### ⚠️ 중요 포인트

- **모델 이름 필수**: CohereRerank 사용 시 반드시 `model` 파라미터 지정
- **API 호출**: Reranker 사용 시 Cohere API 호출 발생 (무료 한도 1,000회/월)
- **성능 개선**: 일반적으로 20-30% 정확도 향상 효과

이제 실제로 두 방식의 차이점을 비교해봅시다! 🚀

In [ ]:
# Cohere Reranker 사용을 위한 라이브러리 import
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever  # 압축 검색기
from langchain_cohere import CohereRerank  # Cohere 재정렬 모델

# 1단계: Cohere Reranker 모델 설정
# rerank-multilingual-v3.0: 최신 다국어 재정렬 모델 사용
compressor = CohereRerank(model="rerank-multilingual-v3.0")

# 2단계: 문맥 압축 검색기 설정
# 기본 검색기(retriever)와 재정렬 모델(compressor)을 결합
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,  # 재정렬을 담당하는 Cohere 모델
    base_retriever=retriever     # 기본 FAISS 검색기
)

# 3단계: 같은 질의문으로 재정렬된 검색 수행
# 이번에는 Cohere Reranker가 결과를 재정렬합니다
compressed_docs = compression_retriever.invoke("Word2Vec 에 대해서 알려줘!")

# 4단계: 재정렬된 검색 결과 출력
print("⚡ Cohere Reranker 적용 후 결과:")
pretty_print_docs(compressed_docs)

---

# 📊 결과 분석 및 비교

## 🔍 검색 결과 차이점 비교

위의 두 결과를 비교해보면 **Cohere Reranker의 효과**를 명확히 확인할 수 있습니다!

### 📈 개선 효과

#### ✅ **관련성 향상**
- **기본 검색**: 키워드 매칭 위주의 단순한 결과
- **Reranker 적용**: 질문의 의도와 맥락을 고려한 정교한 결과

#### ✅ **정확도 개선**
- 질문과 **더 관련성 높은 문서**가 상위에 배치
- **노이즈 문서들**이 하위로 이동하거나 제거

#### ✅ **사용자 경험 향상**
- 첫 번째 결과만 봐도 **원하는 답변**을 찾을 가능성 증가
- 불필요한 문서 탐색 시간 단축

## 💡 활용 팁

### 🎯 언제 Reranker를 사용할까?

**✅ 추천하는 상황:**
- **정확성이 중요한 QA 시스템**
- **전문 분야 문서 검색** (의료, 법률, 기술 등)
- **다국어 검색**이 필요할 때
- **사용자 만족도**를 높이고 싶을 때

**⚠️ 주의할 상황:**
- **응답 속도**가 절대적으로 중요할 때
- **대용량 실시간 처리**가 필요할 때 (API 한도 고려)
- **비용 최적화**가 우선일 때

### 🚀 성능 최적화 방법

1. **기본 검색 개수 조절**: `k=10` → `k=20` (더 많은 후보 중에서 재정렬)
2. **하이브리드 사용**: 중요한 쿼리만 Reranker 적용
3. **캐싱 활용**: 동일한 쿼리의 경우 결과 재사용

## 🎉 마무리

**Cohere Reranker**는 **검색 품질을 한 단계 높여주는 강력한 도구**입니다! 

- 🎯 **간편한 적용**: 몇 줄의 코드로 검색 품질 개선
- 🌍 **다국어 지원**: 한국어를 포함한 100개 이상 언어 지원  
- 💰 **비용 효율적**: 월 1,000회 무료 사용량 제공
- ⚡ **즉시 효과**: 적용 즉시 검색 결과 개선 확인 가능

다음 프로젝트에서 Cohere Reranker를 활용하여 더 똑똑한 검색 시스템을 만들어보세요! 🚀